EDA와 전처리는 주피터노트북에있음

In [1]:
import matplotlib

import matplotlib.pyplot as plt

import seaborn as sns

import pandas as pd

import lightgbm as lgb

import numpy as np

In [4]:
train = pd.read_csv("movies_train.csv")

In [5]:
test = pd.read_csv("movies_test.csv")

In [6]:
train['dir_prev_bfnum'].fillna(0, inplace = True)
test['dir_prev_bfnum'].fillna(0, inplace = True)

In [7]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000) #random_state : 랜덤해야될 값을 미리 정해줌. 다른 pc에서도 같은 값을 넣으면 출력이 같아짐, n_estimators : 반복수

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() 
train['genre'] = le.fit_transform(train['genre']) #장르를 한글에서 숫자로 바꿔주기

In [9]:
train['genre']

0      10
1       2
2      10
3      11
4      11
       ..
595     4
596     4
597     1
598     2
599    10
Name: genre, Length: 600, dtype: int64

In [10]:
test['genre'] = le.transform(test['genre']) #fit_transform이 아닌 이유는 그건 학습하면서 바꾸는 것이기 때문. 이미 학습 완료했으니 그냥 바꿈

In [11]:
test['genre']

0       2
1       5
2       4
3      10
4       1
       ..
238     4
239     3
240     2
241     4
242     5
Name: genre, Length: 243, dtype: int64

In [12]:
features = ['time', 'dir_prev_num', 'num_staff', 'num_actor','dir_prev_bfnum', 'genre']
target = ['box_off_num']

In [13]:
X_train, X_test, y_train = train[features], test[features], train[target]

In [14]:
from sklearn.model_selection import GridSearchCV

In [17]:
from sklearn.model_selection import KFold

In [18]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=777)

In [19]:
params = {
    'learning_rate': [0.1, 0.01, 0.003],
    'min_child_samples': [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid=params,
                  scoring= 'neg_mean_squared_error',
                  cv = k_fold)

In [20]:
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             estimator=LGBMRegressor(n_estimators=1000, random_state=777),
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                         'min_child_samples': [20, 30]},
             scoring='neg_mean_squared_error')

In [21]:
gs.best_params_

{'learning_rate': 0.003, 'min_child_samples': 30}

In [22]:
model = lgb.LGBMRegressor(random_state=777,n_estimators=1000, learning_rate=0.003, min_child_samples=30)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.56673e+12
[200]	valid_0's l2: 2.45583e+12
[300]	valid_0's l2: 2.42575e+12
[400]	valid_0's l2: 2.43392e+12
Early stopping, best iteration is:
[335]	valid_0's l2: 2.42348e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 4.89194e+12
[200]	valid_0's l2: 4.40922e+12
[300]	valid_0's l2: 4.19146e+12
[400]	valid_0's l2: 4.05951e+12
[500]	valid_0's l2: 3.96931e+12
[600]	valid_0's l2: 3.91727e+12
[700]	valid_0's l2: 3.88462e+12
[800]	valid_0's l2: 3.87695e+12
[900]	valid_0's l2: 3.87088e+12
Early stopping, best iteration is:
[876]	valid_0's l2: 3.87035e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.14361e+12
[200]	valid_0's l2: 2.79286e+12
[300]	valid_0's l2: 2.59302e+12
[400]	valid_0's l2: 2.47608e+12
[500]	valid_0's l2: 2.40386e+12
[600]	valid_0's l2: 2.36407e+12
[700]	valid_0's l2: 2.38505e+12
Early stopping, best iteration is:
[6

In [23]:
preds = []

for model in models:
    preds.append(model.predict(X_test))

In [27]:
submission = pd.read_csv("submission.csv")

In [28]:
gslgbm = submission.copy()

In [30]:
import numpy as np

In [31]:
gslgbm['box_off_num'] = np.mean(preds, axis=0)
gslgbm

,title,box_off_num
0,용서는 없다,2.974960e+06
1,아빠가 여자를 좋아해,9.823131e+05
2,하모니,1.283210e+06
3,의형제,1.681758e+06
4,평행 이론,9.095845e+05
...,...,...
238,해에게서 소년에게,7.886130e+04
239,울보 권투부,1.276022e+05
240,어떤살인,4.470473e+05
241,말하지 못한 비밀,2.762433e+05


In [33]:
gslgbm.to_csv('gslgbm.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#이거 왜생겼음?